# (1) Importing/Storing data

In [1]:
import wfdb
import os
import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
### sampled at 250 samples per second

In [3]:
records_A = []
properties_A = []
annot_A = []
AnnSymb_A = []
AnnSamp_A = []
AnnRhythm_A = []
Rpeak_Samp_A = []
Rpeak_Symb_A = []


for f in glob.glob('C:/Users/masud/Skrivebord/Github_Bachelor_Project/wfdb-python/data/arrhythmia_data/*.dat'): ##### change the path to own directory.       
    sig, fields = wfdb.rdsamp(f[:-4], channels=[1])  #### In this function, pass "channels=[0]" or "channels=[1]" to select channel 1 or 2.
    
    ann = wfdb.rdann(f[:-4], 'atr')
    QRS = wfdb.rdann(f[:-4], 'xws')
    Symb = pd.Series(ann.symbol)
    Samp = pd.Series(ann.sample)
    
    QRS_Symb = pd.Series(QRS.symbol)
    QRS_Samp = pd.Series(QRS.sample)
    Rhythm = pd.Series(ann.aux_note)
    records_A.append(sig)
    properties_A.append(fields)
    annot_A.append(ann)
    AnnSymb_A.append(Symb)
    AnnSamp_A.append(Samp)
    Rpeak_Symb_A.append(QRS_Symb)
    Rpeak_Samp_A.append(QRS_Samp)
    AnnRhythm_A.append(Rhythm)
    
    

AnnSymb_A = pd.Series(AnnSymb_A).values
AnnSamp_A = pd.Series(AnnSamp_A).values

In [4]:
AnnSymb_A[47] #Rpeak_Symbol

0       +
1       N
2       N
3       N
4       N
       ..
2759    N
2760    N
2761    N
2762    N
2763    N
Length: 2764, dtype: object

In [5]:
AnnSamp_A[47] #Rpeak_Sample

0           52
1          135
2          366
3          606
4          841
         ...  
2759    648797
2760    649040
2761    649292
2762    649536
2763    649772
Length: 2764, dtype: int32

In [6]:
Rpeak_Symb_A[47] #Annotaion_Symbol

0       +
1       /
2       Q
3       j
4       A
5       u
6       L
7       +
8       ^
9       u
10      ~
11      j
12      A
13      j
14      B
15      ^
16      A
17      p
18    NaN
19      *
20      *
21    NaN
22      =
23      |
24    NaN
25      A
26      j
27    NaN
28      j
29      A
30      p
31      t
32      u
33      +
34      A
35      T
36      D
37    NaN
38      D
39      D
40      s
41      A
42      j
dtype: object

In [7]:
Rpeak_Samp_A[47] #Annotaion_Sample

0       557
1      1101
2      1152
3      1418
4      1515
5      1612
6      2238
7      2283
8      2315
9      2431
10     3055
11     3870
12     4186
13     4487
14     4778
15     4877
16     4988
17     5362
18     5732
19     6534
20     7115
21     7457
22     8307
23     8641
24     8974
25     9008
26     9309
27     9371
28     9660
29    10473
30    11342
31    12214
32    12758
33    13367
34    13940
35    14776
36    15099
37    15432
38    15510
39    15861
40    16199
41    16233
42    16534
dtype: int32

In [8]:
AnnRhythm_A[47]

0       (N 
1          
2          
3          
4          
       ... 
2759       
2760       
2761       
2762       
2763       
Length: 2764, dtype: object

# (2) Pre processing data

In [9]:
labeled_Rpeaks_A = []
appended_data_A = []
for i in range(48): ## 48 records
    for j in range(len(AnnSamp_A[i])-1): ## AnnSamp or AnnRhythm -- both same dimensions
        df = pd.DataFrame(Rpeak_Samp_A[i][(Rpeak_Samp_A[i] > AnnSamp_A[i][j]) & (Rpeak_Samp_A[i] < AnnSamp_A[i][j+1])])
        df['Label'] = AnnRhythm_A[i][j]
        appended_data_A.append(df)
        

labeled_Rpeaks_A = pd.concat(appended_data_A)

In [13]:
labeled_Rpeaks_A

0 Label
0     557      
1     845      
2     893      
3    1159      
4    1256      
..    ...   ...
38  15510      
39  15861      
40  16199      
41  16233      
42  16534      

[2059 rows x 2 columns]

### (below) No annotations of '(N' or '(AFIB' string convention?

In [15]:
labeled_Rpeaks_N_AFIB = labeled_Rpeaks_A.loc[(labeled_Rpeaks_A.Label == '(N') | (labeled_Rpeaks_A.Label == '(AFIB'), :]
labeled_Rpeaks_N_AFIB

Empty DataFrame
Columns: [0, Label]
Index: []